In [ ]:
# Install necessary libraries
!pip install transformers nltk sacrebleu

# Import required libraries
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import time
import nltk
from nltk.translate.meteor_score import meteor_score
import tkinter as tk
from tkinter import messagebox
from google.colab import output

In [ ]:
# Download necessary NLTK data
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# Load the NLLB model for translation
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-1.3B")

Loading tokenizer...


In [ ]:
print("Loading model (this may take a few minutes)...")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-1.3B")

Loading model (this may take a few minutes)...


In [ ]:
# Create translation pipeline
# The NLLB model uses language codes: 'eng_Latn' for English and 'ben_Beng' for Bengali
translator = pipeline("translation", model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang="ben_Beng")

Device set to use cuda:0


In [ ]:
# Function to compute METEOR score
def calculate_meteor(reference, hypothesis):
    """Calculate METEOR score between reference and hypothesis."""
    try:
        # Split into tokens for METEOR calculation
        reference_tokens = reference.split()
        hypothesis_tokens = hypothesis.split()
        score = meteor_score([reference_tokens], hypothesis_tokens)
        return score
    except Exception as e:
        print(f"Error calculating METEOR score: {e}")
        return None

In [ ]:
# Function for user input translation with two reference translations
def translate_user_input():
    print("\n--- User Input Translation with Multiple References ---\n")
    print("For each English input, two reference Bengali translations needed")
    print("The function will calculate METEOR scores for each reference and their average")

    while True:
        # Get user input
        user_input = input("\nEnter English text to translate to Bengali (or type 'exit' to quit): ")

        if user_input.lower() == 'exit':
            print("Exiting translation mode.")
            break

        # Step 1: Translate the user input using the model
        print("\nStep 1: Translating input text...")
        translation = translator(user_input, max_length=256)
        bengali_text = translation[0]['translation_text']

        print(f"\nEnglish: {user_input}")
        print(f"Model Translation: {bengali_text}")

        # Step 2: Get the two required reference translations
        print("\nStep 2: Collecting reference translations...")
        reference1 = input("Enter 1st reference Bengali translation: ")
        reference2 = input("Enter 2nd reference Bengali translation: ")

        # Step 3: Calculate METEOR score for first reference
        print("\nStep 3: Calculating METEOR score for 1st reference...")
        meteor1 = calculate_meteor(reference1, bengali_text)
        if meteor1 is not None:
            print(f"Reference 1: {reference1}")
            print(f"METEOR Score 1: {meteor1:.4f}")
        else:
            print("Could not calculate METEOR score for the 1st reference.")
            meteor1 = 0

        # Step 4: Calculate METEOR score for second reference
        print("\nStep 4: Calculating METEOR score for 2nd reference...")
        meteor2 = calculate_meteor(reference2, bengali_text)
        if meteor2 is not None:
            print(f"Reference 2: {reference2}")
            print(f"METEOR Score 2: {meteor2:.4f}")
        else:
            print("Could not calculate METEOR score for the 2nd reference.")
            meteor2 = 0

        # Step 5: Calculate average METEOR score
        print("\nStep 5: Calculating average METEOR score...")
        if meteor1 is not None and meteor2 is not None:
            avg_meteor = (meteor1 + meteor2) / 2
            print(f"Average METEOR Score: {avg_meteor:.4f}")

            # Additional information: Show the calculation
            print(f"Calculation: ({meteor1:.4f} + {meteor2:.4f}) / 2 = {avg_meteor:.4f}")
        else:
            print("Could not calculate average METEOR score due to errors.")

        print("\n" + "-"*50)

In [ ]:
# Calculate average METEOR score
meteor_scores = []

# Call the function for user input translation
print("\nEnter your own English text for translation to Bangla.")
translate_user_input()

print("\nTranslation session complete!")


Enter your own English text for translation to Bangla.

--- User Input Translation with Multiple References ---

For each English input, you'll provide two reference Bengali translations
The function will calculate METEOR scores for each reference and their average

Enter English text to translate to Bengali (or type 'exit' to quit): He kicked the bucket just days before he was supposed to receive his lifetime achievement award.

Step 1: Translating input text...

English: He kicked the bucket just days before he was supposed to receive his lifetime achievement award.
Model Translation: তিনি তার জীবনকালের পুরস্কার পাওয়ার কয়েকদিন আগে বালতিটি টুকরো টুকরো করে ফেলেছিলেন।

Step 2: Collecting reference translations...
Enter first reference Bengali translation: তিনি আজীবন সম্মাননা পাওয়ার কয়েকদিন আগেই মারা যান।
Enter second reference Bengali translation: তিনি আজীবন সম্মাননা পাওয়ার কয়েকদিন আগেই বালতিটি টুকরো টুকরো করে ফেলেছিলেন। 

Step 3: Calculating METEOR score for first reference...
R